# Using Tools


In [1]:
import instructor
import requests
import json
from pydantic import BaseModel, Field
from instructor import OpenAISchema
from openai import OpenAI

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

#--------------------------------------------------------
# Initialize the OpenAI client for Ollama   
# And add instructor wrapper for Pydantic validation
#--------------------------------------------------------
client = OpenAI(
    base_url="http://localhost:11434/v1/", 
    api_key=""
)
client = instructor.patch(client)

#----------------------------------------------------------
# Tool 
#----------------------------------------------------------
class GetWeather(OpenAISchema):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

    def run(self) -> dict:
        """Fetch weather info from Open-Meteo"""
        response = requests.get(
            f"https://api.open-meteo.com/v1/forecast"
            f"?latitude={self.latitude}&longitude={self.longitude}"
            f"&current=temperature_2m,wind_speed_10m"
        )
        data = response.json()
        return data["current"]

tool_schema = GetWeather.openai_schema
tool_schema["strict"] = True
tools = [{
    "type": "function", 
    "function":GetWeather.openai_schema
}]

#--------------------------------------------------------
# Tool response model 
#--------------------------------------------------------
class WeatherResponse(BaseModel):
    temperature: float = Field(description="The current temperature in celsius for the given location.")
    response: str = Field(description="A natural language response to the user's question.")


In [2]:
model="llama3.1:8b"
messages=[            
    {"role": "system",  "content": "You are a helpful weather assistant."},
    {"role": "user",    "content": "What's the weather like now in Tainan, Taiwan now ?"},
]

#---------------------------------------------------------------------------------------------
# 這是 OpenAI API 官方標準的端點 ===> 用於創建聊天對話的補全 (chat completion)。
#---------------------------------------------------------------------------------------------
# 1. 根據 messages 和 tools，模型決定是要直接生成文字回覆，還是要呼叫工具 (tool call)。
#    當模型決定呼叫工具時，它的回覆會包含工具名稱和參數，但不會執行工具。
#---------------------------------------------------------------------------------------------
completion2 = client.chat.completions.create(
    model=model,    
    messages=messages,     
    temperature=0, 
    tools=tools,      
)

print(f"\nchat completion ===>\n", json.dumps(completion2.model_dump(), indent=2))   

#----------------------------------------------------------------------------
# 2. 如果 LLM 說要 Call Tool
#----------------------------------------------------------------------------
tool_calls = completion2.choices[0].message.tool_calls
if tool_calls:  
    for tool_call in completion2.choices[0].message.tool_calls:
        # 3. 取出 Tool 的 name 和 參數    
        name = tool_call.function.name    
        args = json.loads(tool_call.function.arguments)        
        #print(f"\nCalling function: {name} with args: {args}")               
        #print(f"\nOriginal human message:", messages)    
            
        # 4. 把模型的回覆加入到 messages 列表中。使 messages 現在包含了使用者發問和模型的回覆     
        messages.append(completion2.choices[0].message)    
        print(f"\nMessages after create: ===>\n", completion2.choices[0].message)     
              
        # 5. Tool Calling
        result = None
        if name == "GetWeather":
            #result = get_weather(**args)               
            result = GetWeather(**args).run()        
            messages.append(
                {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
            )  
            print(f"\nTool Calling 結果 ===>\n {result}")           
            


chat completion ===>
 {
  "id": "chatcmpl-288",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_bq48apgg",
            "function": {
              "arguments": "{\"latitude\":22.98,\"longitude\":120.21}",
              "name": "GetWeather"
            },
            "type": "function",
            "index": 0
          }
        ]
      }
    }
  ],
  "created": 1757145406,
  "model": "llama3.1:8b",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_ollama",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 203,
    "total_tokens": 230,
    "completion_tokens_details": {
      "accepted_prediction_tokens": null,
      "audio_tokens": 0,
      "reasoning_to

In [3]:
#----------------------------------------------------------------------------
# 6. 最後讓 LLM 根據工具回傳的內容，生成你想要的自然語言回應。
#    而且回傳格式會符合你定義的 WeatherResponse Pydantic schema
#----------------------------------------------------------------------------
completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

final_response = completion.choices[0].message.parsed
print("\nWeatherResponse Model 定義的 temperature ===> ", final_response.temperature)
print("WeatherResponse Model 定義的 Response ===> ",final_response.response)



WeatherResponse Model 定義的 temperature ===>  30.2
WeatherResponse Model 定義的 Response ===>  Now in Tainan, Taiwan, it is approximately 7:45 AM, with a temperature of around 30.2°C (86.36°F), and the wind speed is about 11.9 m/s (26.71 mph).
